winget install 'Microsoft .NET SDK 7.0'  #Needed. Doesn't work on .Net 8
### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/2/2023 9:00:01 PM
Boot                  Sat 12.2.23 10:36
CpuUsage%             76
RamUsage%             {51}
C:%Free               {55}



This shows which processes are consuming the most cpu, memory and network and have the most number of handles and threads currently on the system.

In [13]:
$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads
# Sort for each property separately
$sortedByCPU = $objArr | Sort-Object CPU -Descending | Select-Object -First 10
$sortedByWS = $objArr | Sort-Object WS -Descending | Select-Object -First 10
$sortedByHandles = $objArr | Sort-Object HandleCount -Descending | Select-Object -First 10
$sortedByThreads = $objArr | Sort-Object Threads -Descending | Select-Object -First 10

$Network = (Get-Counter '\Process(*)\IO Read Operations/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"})|
                sort cookedvalue -Descending|Select -Property instancename -First 10

$harray = for ($j = 0; $j -lt 10; $j++) {
    [PSCustomObject][ordered]@{
        'CPU'     = $sortedByCPU[$j].Process
        'WS'      = $sortedByWS[$j].Process
        'Handles' = $sortedByHandles[$j].Process
        'Threads' = $sortedByThreads[$j].Process
        'Network' = $Network[$j].InstanceName
    }
}
$harray | Format-Table -AutoSize


CPU                    WS                      Handles                Threads                 Network
---                    --                      -------                -------                 -----
msedge►7464             msedge►7464             System►4                System►4                windowsazureguestagent
OneDrive►11916          Grammarly.Desktop►12308 msedge►7464             Memory Compression►2632 dwm
Grammarly.Desktop►12308 SearchHost►6964         explorer►6532           SearchHost►6964         svchost 
Code - Insiders►5628    dwm►772                 lsass►904               explorer►6532           msedge 
Code - Insiders►3908    Code - Insiders►5628    MonitoringHost►4256     msedge►7464             msedge
msedge►6124             msedge►6124             Grammarly.Desktop►12308 Code - Insiders►12832   msedge
explorer►6532           MsMpEng►4004            SearchHost►6964         msedgewebview2►14164    msedge
Code - Insiders►9460    msedge►9492             svchost►120 